In [ ]:
## the is the pre train llm model base on odia language
## this model follow llama2 and gpt architecture

In [1]:
## tiktoken use for use python builtent tokenizer


!pip install torch
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.8 MB/s eta 0:00:00


In [ ]:
# load the odia language data set
with open("odia_data_ind.txt", "r") as f:
    odia_language_data = f.read()



In [2]:
## in this model use gpt2 builten BPE TOKENIZER with 50,000 vocab size
## use 2 special token sos and eos

import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
text=("ବାକ୍ୟ ବିଚାର</EOS>ଏମିତି ଗୋଟିଏ ଦିନ ଯାଉନି ଯେଉଁ ଦିନ ଆର୍ଟିଓ ଅଫିସରେ ଲମ୍ବା ଧାଡ଼ି ଦେଖିବାକୁ ମିଳୁନି ।")

In [ ]:
integers = tokenizer.encode(text, allowed_special={"<SOS>","</EOS>"})

print(integers)

[156, 105, 105, 156, 105, 122, 156, 105, 243, 156, 255, 235, 156, 255, 253, 220, 156, 105, 105, 156, 105, 123, 156, 105, 248, 156, 105, 122, 156, 105, 108, 3556, 36, 2640, 29, 156, 105, 237, 156, 105, 106, 156, 105, 123, 156, 105, 97, 156, 105, 123, 220, 156, 105, 245, 156, 255, 233, 156, 105, 253, 156, 105, 123, 156, 105, 237, 220, 156, 105, 99, 156, 105, 123, 156, 105, 101, 220, 156, 105, 107, 156, 105, 122, 156, 105, 231, 156, 105, 101, 156, 105, 123, 220, 156, 105, 107, 156, 255, 229, 156, 105, 231, 156, 105, 223, 220, 156, 105, 99, 156, 105, 123, 156, 105, 101, 220, 156, 105, 228, 156, 105, 108, 156, 255, 235, 156, 105, 253, 156, 105, 123, 156, 105, 241, 220, 156, 105, 227, 156, 105, 104, 156, 105, 123, 156, 105, 116, 156, 105, 108, 156, 255, 229, 220, 156, 105, 110, 156, 105, 106, 156, 255, 235, 156, 105, 105, 156, 105, 122, 220, 156, 105, 100, 156, 105, 122, 156, 105, 94, 156, 45539, 156, 105, 123, 220, 156, 105, 99, 156, 255, 229, 156, 105, 244, 156, 105, 123, 156, 105, 105, 15

In [ ]:
print(tokenizer.decode(integers))

ବାକ୍ୟ ବିଚାର</EOS>ଏମିତି ଗୋଟିଏ ଦିନ ଯାଉନି ଯେଉଁ ଦିନ ଆର୍ଟିଓ ଅଫିସରେ ଲମ୍ବା ଧାଡ଼ି ଦେଖିବାକୁ ମିଳୁନି ।


In [3]:
## now impliment sliding window for self-supervise learning
## next word prediction
## use data loader


import torch
from torch.utils.data import Dataset, DataLoader

In [4]:
class ODIA_Dataset(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"</EOS>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [5]:
def create_dataloader(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = ODIA_Dataset(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [6]:
## test our data loader
test_data=("ଆଇସିସି ବିଶ୍ବକପରେ ସେ ଅଂଶ ଗ୍ରହଣ କରିଥିବାରୁ ସେ ନିଜକୁ ଭାଗ୍ୟଶାଳୀ ମନେ କରୁଛନ୍ତି ।କରୀନା କପୂର ହେଉଛନ୍ତି ଭାରତୀୟ କ୍ରିକେଟ ଟିମର ପୂର୍ବତନ ଅଧିନାୟକ ମସୁର ଅଲ୍ଲୀ ଖାନଙ୍କ ପଟ୍ଟୌଦୀଙ୍କ ବୋହୂ ।କିନ୍ତୁ ଏସବୁ ସତ୍ତ୍ବେ ଆରଟିଓ ଅଫିସରେ ଏବେ ଲୋକଙ୍କ ଆଖିରେ ଆଙ୍ଗୁଠି ଗେଞ୍ଜି ଆବେଦନକାରୀ ପିଛା ୨୨ଟଙ୍କା ଲୁଟୁଛନ୍ତି ।ଗୁଗଲ ସୂଚନା ଅନୁଯାୟୀ ପରବର୍ତ୍ତୀ ମୂହୁର୍ତ୍ତରେ ଜୋସେଫ ପ୍ଲେଟ୍ୟୁ ନିଜ ଆଖିର ଜ୍ୟୋତି ହରାଇ ବସିଥିଲେ ")

dataloader = create_dataloader(
    test_data, batch_size=1, max_length=10, stride=4, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[156, 105, 228, 156, 105, 229, 156, 105, 116, 156]]), tensor([[105, 228, 156, 105, 229, 156, 105, 116, 156, 105]])]


'/content/drive/MyDrive/odia_data.txt'

In [ ]:
dataloader = create_dataloader(test_data, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[156, 105, 228, 156],
        [105, 229, 156, 105],
        [116, 156, 105, 123],
        [156, 105, 116, 156],
        [105, 123, 220, 156],
        [105, 105, 156, 105],
        [123, 156, 105, 114],
        [156, 255, 235, 156]])

Targets:
 tensor([[105, 228, 156, 105],
        [229, 156, 105, 116],
        [156, 105, 123, 156],
        [105, 116, 156, 105],
        [123, 220, 156, 105],
        [105, 156, 105, 123],
        [156, 105, 114, 156],
        [255, 235, 156, 105]])


In [ ]:
## now apply word emmbeding
## using pytorch builten emmbeding

voc_size=50
emmbed_dim=10

In [ ]:
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(voc_size, emmbed_dim)

In [ ]:
embedding_layer(torch.tensor(1))

tensor([ 0.6984, -1.4097,  0.1794,  1.8951,  0.4954,  0.2692, -0.0770, -1.0205,
        -0.1690,  0.9178], grad_fn=<EmbeddingBackward0>)

In [ ]:
## 10 embeding

## test with 50,000 vocab 256 emmbed
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [ ]:
max_length = 40
dataloader = create_dataloader(
    odia_language_data, batch_size=10, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [ ]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([10, 40, 256])


In [ ]:
## position embeding
pos_embedding_layer = torch.nn.Embedding(10, 256)

In [ ]:
pos_embeddings = pos_embedding_layer(torch.arange(10))
print(pos_embeddings.shape)

torch.Size([10, 256])


In [7]:

## now apply multi head mask self attntion
import torch.nn as nn
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        # Reduce the projection dim to match desired output dim
        self.head_dim = d_out // num_heads

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        # Linear layer to combine head outputs
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # Shape: (b, num_tokens, d_out)
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        # implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

In [8]:
## create swiglu activation for hiden neural network layer
import torch.nn.functional as F
class SwiGLU(nn.Module):
    def forward(self, x):
        x, gate = x.chunk(2, dim=-1)
        return F.silu(gate) * x

In [9]:
## create gelu activation for hiden neural network layer
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))

In [10]:
## apply RMSNorm for layer normalization
## it help reduce the computational

class RMSNorm(nn.Module):
    def __init__(self, d, p=-1., eps=1e-8, bias=False):

        super(RMSNorm, self).__init__()

        self.eps = eps
        self.d = d
        self.p = p
        self.bias = bias

        self.scale = nn.Parameter(torch.ones(d))
        self.register_parameter("scale", self.scale)

        if self.bias:
            self.offset = nn.Parameter(torch.zeros(d))
            self.register_parameter("offset", self.offset)

    def forward(self, x):
        if self.p < 0. or self.p > 1.:
            norm_x = x.norm(2, dim=-1, keepdim=True)
            d_x = self.d
        else:
            partial_size = int(self.d * self.p)
            partial_x, _ = torch.split(x, [partial_size, self.d - partial_size], dim=-1)

            norm_x = partial_x.norm(2, dim=-1, keepdim=True)
            d_x = partial_size

        rms_x = norm_x * d_x ** (-1. / 2)
        x_normed = x / (rms_x + self.eps)

        if self.bias:
            return self.scale * x_normed + self.offset

        return self.scale * x_normed

In [ ]:
torch.manual_seed(123)
batch_example = torch.randn(2, 5)

In [11]:
## feedforward network with 2 layer 3,144 first layer and 786 output layer

class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

In [12]:
MODEL_CONFIG = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 9,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [ ]:
ffn = FeedForward(MODEL_CONFIG)

# input shape: [batch_size, num_token, emb_size]
x = torch.rand(2, 3, 768)
out = ffn(x)
print(out.shape)

torch.Size([2, 3, 768])


In [13]:
## transformer block
## combine multihead attention layer normalization feedforward network
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = RMSNorm(cfg["emb_dim"])
        self.norm2 = RMSNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x

In [14]:
torch.manual_seed(123)

x = torch.rand(2, 4, 768)  # Shape: [batch_size, num_tokens, emb_dim]
block = TransformerBlock(MODEL_CONFIG)
output = block(x)

print("Input shape:", x.shape)
print("Output shape:", output.shape)

Input shape: torch.Size([2, 4, 768])
Output shape: torch.Size([2, 4, 768])


In [15]:

class ODIA_Model(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        self.final_norm = RMSNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

In [16]:
tokenizer = tiktoken.get_encoding("gpt2")

batch = []

txt1 = "ଆଇସିସି ବିଶ୍ବକପରେ ସେ ଅଂଶ ଗ୍ରହଣ କରିଥିବାରୁ ସେ ନିଜକୁ ଭାଗ୍ୟଶାଳୀ ମନେ କରୁଛନ୍ତି "
txt2 = "କରୀନା କପୂର ହେଉଛନ୍ତି ଭାରତୀୟ କ୍ରିକେଟ ଟିମର ପୂର୍ବତନ ଅଧିନାୟକ ମସୁର ଅଲ୍ଲୀ କରୁଛ "

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[156, 105, 228, 156, 105, 229, 156, 105, 116, 156, 105, 123, 156, 105,
         116, 156, 105, 123, 220, 156, 105, 105, 156, 105, 123, 156, 105, 114,
         156, 255, 235, 156, 105, 105, 156, 105, 243, 156, 105, 103, 156, 105,
         108, 156, 255, 229, 220, 156, 105, 116, 156, 255, 229, 220, 156, 105,
         227, 156, 105, 224, 156, 105, 114, 220, 156, 105, 245, 156, 255, 235,
         156, 105, 108, 156, 105, 117, 156, 105,  96, 220, 156, 105, 243, 156,
         105, 108, 156, 105, 123, 156, 105,  98, 156, 105, 123, 156, 105, 105,
         156, 105, 122, 156, 105, 108, 156, 255, 223, 220, 156, 105, 116, 156,
         255, 229, 220, 156, 105, 101, 156, 105, 123, 156, 105, 250, 156, 105,
         243, 156, 255, 223, 220, 156, 105, 255, 156, 105, 122, 156, 105, 245,
         156, 255, 235, 156, 255, 253, 156, 105, 114, 156, 105, 122, 156, 105,
         111, 156, 255, 222, 220, 156, 105, 106, 156, 105, 101, 156, 255, 229,
         220, 156, 105, 243, 156, 105, 108, 156, 255

In [17]:
torch.manual_seed(123)
model = ODIA_Model(MODEL_CONFIG)

out = model(batch)
print("Input batch:\n", batch)
print("\nOutput shape:", out.shape)
print(out)

Input batch:
 tensor([[156, 105, 228, 156, 105, 229, 156, 105, 116, 156, 105, 123, 156, 105,
         116, 156, 105, 123, 220, 156, 105, 105, 156, 105, 123, 156, 105, 114,
         156, 255, 235, 156, 105, 105, 156, 105, 243, 156, 105, 103, 156, 105,
         108, 156, 255, 229, 220, 156, 105, 116, 156, 255, 229, 220, 156, 105,
         227, 156, 105, 224, 156, 105, 114, 220, 156, 105, 245, 156, 255, 235,
         156, 105, 108, 156, 105, 117, 156, 105,  96, 220, 156, 105, 243, 156,
         105, 108, 156, 105, 123, 156, 105,  98, 156, 105, 123, 156, 105, 105,
         156, 105, 122, 156, 105, 108, 156, 255, 223, 220, 156, 105, 116, 156,
         255, 229, 220, 156, 105, 101, 156, 105, 123, 156, 105, 250, 156, 105,
         243, 156, 255, 223, 220, 156, 105, 255, 156, 105, 122, 156, 105, 245,
         156, 255, 235, 156, 255, 253, 156, 105, 114, 156, 105, 122, 156, 105,
         111, 156, 255, 222, 220, 156, 105, 106, 156, 105, 101, 156, 255, 229,
         220, 156, 105, 243, 156, 105,

In [ ]:
## total para metars
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

Total number of parameters: 141,738,240


In [ ]:
total_params_gpt2 =  total_params - sum(p.numel() for p in model.out_head.parameters())
print(f"Number of trainable parameters considering weight tying: {total_params_gpt2:,}")

Number of trainable parameters considering weight tying: 103,140,864


In [18]:
## now devide data train and test
## use 90% of data for traning purpose and 10% data for validation

## read odia data
with open("/content/odia_data _ind.txt","r") as f:
    odia_language_data = f.read()
train_ratio = 0.90
split_idx = int(train_ratio * len(odia_language_data))
train_data = odia_language_data[:split_idx]
val_data = odia_language_data[split_idx:]


torch.manual_seed(123)

train_loader = create_dataloader(
    train_data,
    batch_size=10,
    max_length=MODEL_CONFIG["context_length"],
    stride=MODEL_CONFIG["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)

val_loader = create_dataloader(
    val_data,
    batch_size=10,
    max_length=MODEL_CONFIG["context_length"],
    stride=MODEL_CONFIG["context_length"],
    drop_last=False,
    shuffle=False,
    num_workers=0
)

In [ ]:
print("Train loader:")
for x, y in train_loader:
    print(x.shape, y.shape)

print("\nValidation loader:")
for x, y in val_loader:
    print(x.shape, y.shape)

Train loader:
torch.Size([4, 1024]) torch.Size([4, 1024])
torch.Size([4, 1024]) torch.Size([4, 1024])
torch.Size([4, 1024]) torch.Size([4, 1024])
torch.Size([4, 1024]) torch.Size([4, 1024])
torch.Size([4, 1024]) torch.Size([4, 1024])
torch.Size([4, 1024]) torch.Size([4, 1024])
torch.Size([4, 1024]) torch.Size([4, 1024])
torch.Size([4, 1024]) torch.Size([4, 1024])
torch.Size([4, 1024]) torch.Size([4, 1024])
torch.Size([4, 1024]) torch.Size([4, 1024])
torch.Size([4, 1024]) torch.Size([4, 1024])
torch.Size([4, 1024]) torch.Size([4, 1024])
torch.Size([4, 1024]) torch.Size([4, 1024])
torch.Size([4, 1024]) torch.Size([4, 1024])
torch.Size([4, 1024]) torch.Size([4, 1024])
torch.Size([4, 1024]) torch.Size([4, 1024])
torch.Size([4, 1024]) torch.Size([4, 1024])
torch.Size([4, 1024]) torch.Size([4, 1024])
torch.Size([4, 1024]) torch.Size([4, 1024])
torch.Size([4, 1024]) torch.Size([4, 1024])
torch.Size([4, 1024]) torch.Size([4, 1024])
torch.Size([4, 1024]) torch.Size([4, 1024])
torch.Size([4, 102

In [ ]:
train_tokens = 0
for input_batch, target_batch in train_loader:
    train_tokens += input_batch.numel()

val_tokens = 0
for input_batch, target_batch in val_loader:
    val_tokens += input_batch.numel()

print("Training tokens:", train_tokens)
print("Validation tokens:", val_tokens)
print("All tokens:", train_tokens + val_tokens)

Training tokens: 520192
Validation tokens: 57344
All tokens: 577536


In [19]:
## calculate losss
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
    return loss


def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # Reduce the number of batches to match the total number of batches in the data loader
        # if num_batches exceeds the number of batches in the data loader
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches

In [20]:
## this function give every word to unique id
def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'</EOS>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor


## this function decode id to word
def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # remove batch dimension
    return tokenizer.decode(flat.tolist())

In [21]:
## train the model
## using adam optimizer
def train_model_simple(model, train_loader, val_loader, optimizer, device, num_epochs,
                       eval_freq, eval_iter, tokenizer):
    # Initialize lists to track losses and tokens seen
    train_losses, val_losses, track_tokens_seen = [], [], []
    tokens_seen, global_step = 0, -1

    # Main training loop
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode

        for input_batch, target_batch in train_loader:
            optimizer.zero_grad() # Reset loss gradients from previous batch iteration
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            loss.backward() # Calculate loss gradients
            optimizer.step() # Update model weights using loss gradients
            tokens_seen += input_batch.numel()
            global_step += 1

            # Optional evaluation step
            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                track_tokens_seen.append(tokens_seen)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")



    return train_losses, val_losses, track_tokens_seen







In [22]:
def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [24]:
torch.manual_seed(123)
model = ODIA_Model(MODEL_CONFIG)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)

num_epochs = 10
train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=5, eval_iter=5,
     tokenizer=tokenizer
)

Ep 1 (Step 000000): Train loss 5.060, Val loss 5.056
Ep 1 (Step 000005): Train loss 2.643, Val loss 2.659
Ep 1 (Step 000010): Train loss 1.926, Val loss 1.920
Ep 1 (Step 000015): Train loss 1.573, Val loss 1.565
Ep 1 (Step 000020): Train loss 1.472, Val loss 1.447
Ep 1 (Step 000025): Train loss 1.412, Val loss 1.402
Ep 1 (Step 000030): Train loss 1.398, Val loss 1.385
Ep 1 (Step 000035): Train loss 1.385, Val loss 1.369
Ep 1 (Step 000040): Train loss 1.373, Val loss 1.365
Ep 1 (Step 000045): Train loss 1.371, Val loss 1.358
Ep 1 (Step 000050): Train loss 1.384, Val loss 1.352
Ep 1 (Step 000055): Train loss 1.366, Val loss 1.347


KeyboardInterrupt: 

In [ ]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx is (batch, n_tokens) array of indices in the current context
    for _ in range(max_new_tokens):

        # Crop current context if it exceeds the supported context size
        # E.g., if LLM supports only 5 tokens, and the context size is 10
        # then only the last 5 tokens are used as context
        idx_cond = idx[:, -context_size:]

        # Get the predictions
        with torch.no_grad():
            logits = model(idx_cond)

        # Focus only on the last time step
        # (batch, n_tokens, vocab_size) becomes (batch, vocab_size)
        logits = logits[:, -1, :]

        # Apply softmax to get probabilities
        probas = torch.softmax(logits, dim=-1)  # (batch, vocab_size)

        # Get the idx of the vocab entry with the highest probability value
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)  # (batch, 1)

        # Append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx

In [ ]:
model.to("cpu")
model.eval()

tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids("କାକା ଆଚନ୍ତ୍ରାଳଭା ଗ୍ରଂଗୋରି", tokenizer),
    max_new_tokens=60,
    context_size=MODEL_CONFIG["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 କାକା ଆଚନ୍ତ୍ରାଳଭା ଗ୍ରଂଗୋରିକକରକରିକ୍ଯରର୍ଯାନ୍କାରା


In [ ]:
torch.save(model.state_dict(), "model_weight.pth")

In [ ]:
model = ODIA_Model(MODEL_CONFIG)
model.to(device)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load("model_weight.pth", map_location=device, weights_only=True))
model.eval();